In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return False
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [4]:
#print(credentials.ACCESS_TOKEN)

In [ ]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @narendramodi: We will always be indebted to Sri Guru Teg Bahadur Ji, Sri Gobind Singh Ji, Mata Gujri Ji and the four Sahibzaades. 

The…
Long: None, Lati: None
RT @Sanju_Verma_: Mahua Moitra,the forever ranting, rabid,rabble rouser from @AITCofficial has audacity to take jibes at #MannkiBaat addres…
Long: None, Lati: None
RT @narendramodi: Delicious in taste, combined with the warmth of the people of Kashmir! 

The Kesar from Kashmir is a fine treat.

Be voca…
Long: None, Lati: None
RT @vijayrupanibjp: Honourable Prime Minister Shri @narendramodi's #MannKiBaat with the nation.

https://t.co/RUQ4mEPZln
Long: None, Lati: None
RT @narendramodi: We will always be indebted to Sri Guru Teg Bahadur Ji, Sri Gobind Singh Ji, Mata Gujri Ji and the four Sahibzaades. 

The…
Long: None, Lati: None
RT @PMOIndia: The next time you want to buy Kesar, do try the Kesar from Kashmir! #MannKiBaat https://t.co/idaQJhslHB
Long: None, Lati: None
RT @mannkibaat: Making India single use plastic free in 202

RT @narendramodi: Tune-in tomorrow at 11 AM. #MannKiBaat https://t.co/knB24XhvQt
Long: None, Lati: None
RT @AnshuSi97917438: #Love4SSR
Dear Honorable Prime Minister Shri @narendramodi ji 
Do we have an answer for her?
We are Still waiting for…
Long: None, Lati: None
RT @sunilozabjp: #MannKiBaat
Kashmir’s saffron is known across the world for its unique quality. PM said that with GI Tag recognition, the…
Long: None, Lati: None
RT @narendramodi: Received a wide range of inputs for this month’s #MannKiBaat. Tune-in. https://t.co/c5wsyxa1Oq
Long: None, Lati: None
RT @sunilozabjp: #MannKiBaat
Bhagvad Gita inspires one to question. Arjun asked Bhagwan Shri Krishna questions, which opened Gita’s knowled…
Long: None, Lati: None
RT @Indrapal_Gupta1: #MannKiBaat
‘Can do’ and ‘Will do’ spirit of the Indian youth
via NaMo App https://t.co/IExMC6MxYg
Long: None, Lati: None
RT @Shazi__786: Prime Minister's on the Shahjahanpur border Man Ki Baat On farmers protesting by playing thali.

Farmer in full

RT @BjpBiplab: The final #MannKiBaat of 2020 and as usual, a lot of letters, inputs and thoughts have been shared.

People have been sharin…
Long: None, Lati: None
RT @narendramodi: We will always be indebted to Sri Guru Teg Bahadur Ji, Sri Gobind Singh Ji, Mata Gujri Ji and the four Sahibzaades. 

The…
Long: None, Lati: None
RT @narendramodi: Is it ever too late to learn?

Ask this amazing gentleman, who learnt computers in his eighties and is learning even more…
Long: None, Lati: None
RT @TheMamuns: #BeAlert FARMERS.

#8thClassTerriblyLiars
#DamnPMLegalTerrorist!
#Corrupt!
#CrookCriminalPM
WORLD'S #CURSE!!!
Is Worse Than…
Long: None, Lati: None
Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendab… https://t.co/gliCD31Nzv
Long: None, Lati: None
RT @narendramodi: Delicious in taste, combined with the warmth of the people of Kashmir! 

The Kesar from Kashmir is a fine treat.

Be voca…
Long: None, Lati: None
RT @PMOIndia: Goo

RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi

RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi

RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @jitu_vaghani: Some incredible stories of incredible Indians championing the cause of clean and green India..

#MannKiBaat https://t.co/…
Long: None, Lati: None
RT @shirodkarhar

RT @narendramodi: India is full of outstanding people who are working diligently towards furthering cleanliness.

During #MannKiBaat today,…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @shirodkarharshu: PM @narendramodi ji in #MannKiBaat v have 2 think how garbage gets on beaches &amp; mountains in d first place; after all,…
Long: None, Lati: None
RT @vijayrupanibjp: Gita teaches us to keep learning 

#MannKiBaat

via NaMo App https://t.co/U3pF2wMEl4
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Yes!

The numerous efforts to keep our public places and tourist places clean are extremely inspiring and commendable. #M…
Long: None, Lati: None
RT @narendramodi: Is it ever too late to learn?
